# 관광지 추천

### 1. 데이터 불러오기

In [ ]:
import pandas as pd

# 데이터 불러오기
tourist_spots_df = pd.read_csv('./최종_data/관광지리스트_데이터.csv')
sentiment_df = pd.read_csv('./최종_data/감정분석_데이터.csv')

### 2-1. 한국어 말뭉치로 Word2Vec 모델 학습

In [ ]:
import os
import json
from gensim.models import Word2Vec
from nltk.tokenize import word_tokenize

# '여행레저' 폴더의 경로
corpus_root = './여행레저'

# 모든 JSON 파일을 읽어서 말뭉치를 만듭니다
sentences = []

for filename in os.listdir(corpus_root):
    if filename.endswith('.json'):
        file_path = os.path.join(corpus_root, filename)
        with open(file_path, 'r', encoding='utf-8') as file:
            data = json.load(file)
            for item in data.get("SJML", {}).get("text", []):  # 각 JSON 객체를 반복
                if 'content' in item:  # 'content' 필드가 있다고 가정
                    text = item['content']
                    tokenized_sentence = word_tokenize(text)
                    sentences.append(tokenized_sentence)

# 만약 sentences 리스트가 비어있지 않은지 확인
if len(sentences) > 0:
    # Word2Vec 모델 초기화
    word2vec_model = Word2Vec(vector_size=300, window=5, min_count=5, workers=4)

    # 어휘 구축
    word2vec_model.build_vocab(sentences)

    # 어휘 크기 출력 (디버깅용)
    print(f"Vocabulary size: {len(word2vec_model.wv)}")

    # 모델 학습
    word2vec_model.train(sentences, total_examples=word2vec_model.corpus_count, epochs=word2vec_model.epochs)
else:
    print("No sentences found for training.")

### 2-2. 학습된 모델 저장 했으니, 향후 꺼내다 쓸때

In [ ]:
from gensim.models import Word2Vec

# 모델 로드
word2vec_model = Word2Vec.load('./여행레저_말뭉치_Word2Vec 모델/여행레저_말뭉치.model')

### 3. 입력 키워드 & 관광지리스트 키워드간 유사도 계산

In [ ]:
# 유사한 키워드를 찾는 함수 (유사도 점수 포함)
def get_most_similar_keywords(input_keyword, keywords, model):
    similarities = []
    
    for keyword in keywords:
        try:
            # 두 키워드 간의 유사도 계산
            similarity = min(model.wv.similarity(input_keyword, keyword), 1.0)
        except KeyError:
            # 만약 키워드가 모델에 없는 경우 유사도를 0으로 설정
            similarity = 0
        similarities.append((keyword, similarity))
    
    # 유사도가 높은 키워드 2개 선택
    top_k = 2
    sorted_similarities = sorted(similarities, key=lambda x: x[1], reverse=True)[:top_k]
    
    return sorted_similarities

# 관광지리스트_데이터에서 키워드 리스트 가져오기
keywords = tourist_spots_df['키워드'].tolist()

# 사용자가 입력한 키워드
input_keyword = input('원하는 여행 키워드를 입력하세요:')

# 가장 유사한 키워드 2개 찾기
similar_keywords = get_most_similar_keywords(input_keyword, keywords, word2vec_model)

# 유사한 키워드와 점수 출력
for keyword, score in similar_keywords:
    print(f"키워드: {keyword}, 유사도 점수: {score:.3f}")

### 4. 사용자가 선택한 키워드의 관광지 리스트 출력

In [ ]:
# 사용자가 입력한 키워드
user_input = input("출력된 두 키워드 중 더 원하는 하나를 입력하세요: ").strip()

# 입력된 키워드에 따라 적절한 항목을 반환
selected_keyword = None
for keyword, score in similar_keywords:
    if user_input == keyword:
        selected_keyword = (keyword, score)
        break

In [ ]:
# 선택한 키워드에 해당하는 관광지 리스트 출력

tourist_spots = tourist_spots_df[tourist_spots_df['키워드'] == selected_keyword[0]]

In [ ]:
# 첫 번째 행의 '관광지 리스트' 컬럼 데이터를 가져와서 ','를 기준으로 분리

tourist_spots_list = tourist_spots_df.iloc[0]['관광지 리스트'].split(',')

### 5. 감정분석 데이터에서 각 관광지의 감정분석 점수 계산

In [ ]:
# 관광지별로 Sentiment_Lemma 평균값 계산

sentiment_scores = []

for spot in tourist_spots_list:
    spot = spot.strip()  # 앞뒤 공백 제거
    # sentiment_df에서 해당 관광지와 일치하는 행을 찾음
    matching_rows = sentiment_df[sentiment_df['관광지'] == spot]
    
    if not matching_rows.empty:
        # Sentiment_Lemma 평균값 계산
        avg_sentiment = matching_rows['Sentiment_Lemma'].mean()
        sentiment_scores.append((spot, avg_sentiment))

### 6-1. 감정분석 점수 TOP3 관광지 출력 (최종 선택 방법)

In [ ]:
# 평균값을 기준으로 내림차순 정렬 후 상위 3개 관광지 선택
top_3_sentiments = sorted(sentiment_scores, key=lambda x: x[1], reverse=True)[:3]

# 결과 출력
for spot, score in top_3_sentiments:
    print(f"관광지: {spot}, 감정분석 평균 점수: {round(score, 3)}")

### 6-2. 감정분석 점수 상위 50% & 키워드와 관광지 이름간의 유사성 고려해서 Top1 관광지 출력

In [ ]:
import numpy as np

# 관광지별로 Sentiment_Lemma 평균값 계산
sentiment_scores = []

for spot in tourist_spots_list:
    spot = spot.strip()  # 앞뒤 공백 제거
    # sentiment_df에서 해당 관광지와 일치하는 행을 찾음
    matching_rows = sentiment_df[sentiment_df['관광지'] == spot]
    
    if not matching_rows.empty:
        # Sentiment_Lemma 평균값 계산
        avg_sentiment = matching_rows['Sentiment_Lemma'].mean()
        sentiment_scores.append((spot, avg_sentiment))

# 평균값을 기준으로 내림차순 정렬
sorted_sentiments = sorted(sentiment_scores, key=lambda x: x[1], reverse=True)

# 상위 50%의 관광지 선택
top_50_percent_cutoff = np.percentile([score for _, score in sorted_sentiments], 50)
top_50_percent_spots = [spot for spot, score in sorted_sentiments if score >= top_50_percent_cutoff]

# 상위 50% 관광지 중에서 키워드와의 유사도를 계산
similarity_scores = []

for spot in top_50_percent_spots:
    try:
        similarity = word2vec_model.wv.similarity(input_keyword, spot)
        similarity_scores.append((spot, similarity))
    except KeyError:
        continue  

# 유사도가 높은 관광지 1개 선택
top_similar_spots = sorted(similarity_scores, key=lambda x: x[1], reverse=True)[:1]

# 결과 출력
for spot, similarity in top_similar_spots:
    print(f"관광지: {spot}, 유사도 점수: {round(similarity, 3)}")